In [1]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np
import scipy.sparse as sp
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

### 数据预处理

In [2]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

In [3]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))  #  矩阵行求和
    r_inv = np.power(rowsum, -1).flatten()  # 求和的-1次方
    r_inv[np.isinf(r_inv)] = 0.   # 如果是inf，转换成0
    r_mat_inv = sp.diags(r_inv)  # 构造对角戏矩阵
    mx = r_mat_inv.dot(mx)  # 构造D-1*A，非对称方式，简化方式
    return mx

In [4]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [5]:
def load_data(path=r"C:\Users\sss\Desktop\cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)  # 取特征feature
    labels = encode_onehot(idx_features_labels[:, -1])  # one-hot label

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)  # 节点
    idx_map = {j: i for i, j in enumerate(idx)}   # 构建节点的索引字典
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),  # 导入edge的数据
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)    # 将之前的转换成字典编号后的边
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),  # 构建边的邻接矩阵
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix，计算转置矩阵。将有向图转成无向图
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)   # 对特征做了归一化的操作
    adj = normalize(adj + sp.eye(adj.shape[0]))   # 对A+I归一化
    
    # 训练，验证，测试的样本
    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)
    # 将numpy的数据转换成torch格式
    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

In [6]:
adj, features, labels, idx_train, idx_val, idx_test = load_data()

Loading cora dataset...


In [11]:
print("# 特征 \t # 标签 \t # 矩阵关系")
print(adj.shape, "\t", features.shape, "\t", labels.shape)

# 特征 	 # 标签 	 # 矩阵关系
torch.Size([2708, 2708]) 	 torch.Size([2708, 1433]) 	 torch.Size([2708])


### 模型准备

In [17]:
class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))  # input_features, out_features
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

#         标准化
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)  # 随机化参数
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)  # GraphConvolution forward。input*weight
        output = torch.spmm(adj, support)  # 稀疏矩阵的相乘，和mm一样的效果
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [18]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()
        self.gc1 = GraphConvolution(nfeat, nhid) # 构建第一层 GCN
        self.gc2 = GraphConvolution(nhid, nclass) # 构建第二层 GCN
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

In [19]:
hidden = 16
dropout = 0.5

model = GCN(nfeat=features.shape[1],
            nhid=hidden,
            nclass=labels.max().item() + 1,
            dropout=dropout)

### 模型参数

In [27]:
seed = 42
cuda = not False and torch.cuda.is_available()
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [21]:
if cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

### 训练和测试

In [23]:
def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

In [30]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad() # GraphConvolution forward
    # 重点输出 ！！！
    output = model(features, adj)   # 运行模型，输入参数 (features, adj)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not False:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], labels[idx_val])
    acc_val = accuracy(output[idx_val], labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))

In [31]:
def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))

In [32]:
t_total = time.time()
for epoch in range(200):
    train(epoch)
print("Optimization Finished")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

Epoch: 0001 loss_train: 1.9396 acc_train: 0.1714 loss_val: 1.9192 acc_val: 0.1567 time: 0.0558s
Epoch: 0002 loss_train: 1.9019 acc_train: 0.2357 loss_val: 1.9034 acc_val: 0.1567 time: 0.0219s
Epoch: 0003 loss_train: 1.8979 acc_train: 0.2571 loss_val: 1.8888 acc_val: 0.1567 time: 0.0840s
Epoch: 0004 loss_train: 1.8899 acc_train: 0.2500 loss_val: 1.8750 acc_val: 0.1600 time: 0.0219s
Epoch: 0005 loss_train: 1.8752 acc_train: 0.2286 loss_val: 1.8621 acc_val: 0.2400 time: 0.0219s
Epoch: 0006 loss_train: 1.8578 acc_train: 0.3571 loss_val: 1.8497 acc_val: 0.4067 time: 0.0219s
Epoch: 0007 loss_train: 1.8447 acc_train: 0.3429 loss_val: 1.8380 acc_val: 0.4600 time: 0.0229s
Epoch: 0008 loss_train: 1.8377 acc_train: 0.3500 loss_val: 1.8268 acc_val: 0.4567 time: 0.0259s
Epoch: 0009 loss_train: 1.8414 acc_train: 0.2714 loss_val: 1.8164 acc_val: 0.4233 time: 0.0289s
Epoch: 0010 loss_train: 1.8040 acc_train: 0.3714 loss_val: 1.8066 acc_val: 0.3767 time: 0.0239s
Epoch: 0011 loss_train: 1.7967 acc_train

Epoch: 0087 loss_train: 0.8495 acc_train: 0.8071 loss_val: 1.0435 acc_val: 0.7733 time: 0.0229s
Epoch: 0088 loss_train: 0.8584 acc_train: 0.7857 loss_val: 1.0354 acc_val: 0.7867 time: 0.0259s
Epoch: 0089 loss_train: 0.7945 acc_train: 0.8286 loss_val: 1.0273 acc_val: 0.7867 time: 0.0349s
Epoch: 0090 loss_train: 0.7463 acc_train: 0.8786 loss_val: 1.0196 acc_val: 0.7967 time: 0.0309s
Epoch: 0091 loss_train: 0.7909 acc_train: 0.8357 loss_val: 1.0124 acc_val: 0.8000 time: 0.0409s
Epoch: 0092 loss_train: 0.8045 acc_train: 0.8429 loss_val: 1.0053 acc_val: 0.8000 time: 0.0469s
Epoch: 0093 loss_train: 0.7821 acc_train: 0.8571 loss_val: 0.9985 acc_val: 0.8033 time: 0.0469s
Epoch: 0094 loss_train: 0.7671 acc_train: 0.8500 loss_val: 0.9922 acc_val: 0.8033 time: 0.0489s
Epoch: 0095 loss_train: 0.7350 acc_train: 0.8500 loss_val: 0.9849 acc_val: 0.8033 time: 0.0429s
Epoch: 0096 loss_train: 0.7821 acc_train: 0.8143 loss_val: 0.9781 acc_val: 0.8000 time: 0.0399s
Epoch: 0097 loss_train: 0.7978 acc_train

Epoch: 0176 loss_train: 0.4452 acc_train: 0.9071 loss_val: 0.7522 acc_val: 0.8067 time: 0.0319s
Epoch: 0177 loss_train: 0.4778 acc_train: 0.8786 loss_val: 0.7496 acc_val: 0.8067 time: 0.0309s
Epoch: 0178 loss_train: 0.4636 acc_train: 0.9143 loss_val: 0.7488 acc_val: 0.8133 time: 0.0379s
Epoch: 0179 loss_train: 0.4658 acc_train: 0.9500 loss_val: 0.7490 acc_val: 0.8100 time: 0.0419s
Epoch: 0180 loss_train: 0.4564 acc_train: 0.9357 loss_val: 0.7487 acc_val: 0.8100 time: 0.0349s
Epoch: 0181 loss_train: 0.4813 acc_train: 0.9286 loss_val: 0.7481 acc_val: 0.8100 time: 0.0309s
Epoch: 0182 loss_train: 0.4271 acc_train: 0.9214 loss_val: 0.7454 acc_val: 0.8100 time: 0.0349s
Epoch: 0183 loss_train: 0.4567 acc_train: 0.9071 loss_val: 0.7416 acc_val: 0.8167 time: 0.0319s
Epoch: 0184 loss_train: 0.3984 acc_train: 0.9357 loss_val: 0.7374 acc_val: 0.8133 time: 0.0309s
Epoch: 0185 loss_train: 0.4377 acc_train: 0.9214 loss_val: 0.7351 acc_val: 0.8200 time: 0.0319s
Epoch: 0186 loss_train: 0.4739 acc_train

In [33]:
test()

Test set results: loss= 0.7576 accuracy= 0.8230
